# ResNet with TensorFlow (REFER TO COLAB)
## An implementation of a Residual Neural Network with transfer learning using classes to classify the mnist handwritten digits database

### Setup

In [1]:
# Import statements
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model, losses, models, layers, datasets
import matplotlib.pyplot as plt

2023-07-05 21:27:52.622550: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.10.0


In [2]:
# Data
fashion_mnist = datasets.mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Scaling images between 0 and 1
x_train, x_test = x_train/255, x_test/255

# Resizing images to be 32x32 because that is the minimum
x_train = tf.pad(x_train, [[0,0], [2,2], [2,2]])
x_test = tf.pad(x_test, [[0, 0], [2,2], [2,2]])

x_train = tf.expand_dims(x_train, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)
x_train = tf.repeat(x_train, 3, axis=3)
x_test = tf.repeat(x_test, 3, axis=3)


# Create a validation set
x_val = x_train[-2000:,:,:]
y_val = y_train[-2000:]
x_train = x_train[:-2000,:,:]
y_train = y_train[:-2000]

# Batching and shuffling the dataset (Marked for review)
# train_ds = tf.data.Dataset.from_tensor_slices(
#     (x_train, y_train)).shuffle(10000).batch(32)
# test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

2023-07-05 21:28:09.185457: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Building the Model

In [3]:
# Base Model
base_model = tf.keras.applications.ResNet152(weights="imagenet", include_top = False, input_shape = (32,32,3))

for layer in base_model.layers:
    layer.trainable = False

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = Flatten()(base_model.output)
        self.d1 = Dense(1024, activation="relu")
        self.d2 = Dense(10)
    
    def call(self, x):
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()
predictions = model


In [4]:
# Specific Code
x = Flatten()(base_model.output)
x = Dense(1000, activation="relu")(x)
predictions = layers.Dense(10, activation = "softmax")(x)

In [5]:
head_model = Model(inputs = base_model.input, outputs=predictions)
head_model.compile(optimizer="adam", loss=losses.sparse_categorical_crossentropy, metrics=["accuracy"])

### Running the Model

In [6]:
history = head_model.fit(x_train, y_train, batch_size=128, epochs=30, validation_data=(x_val, y_val))
# Could not run fast enough on personal computer, thus this line was run on google colab for increased computational power.

Epoch 1/30
454/454 [==============================] - 315s 666ms/step - loss: 1.0225 - accuracy: 0.6671 - val_loss: 0.6199 - val_accuracy: 0.8055
Epoch 2/30
153/454 [=========>....................] - ETA: 3:02 - loss: 0.6886 - accuracy: 0.7770

KeyboardInterrupt: 

In [ ]:
head_model.evaluate(x_test, y_test)

313/313 [==============================] - 18s 55ms/step - loss: 0.2853 - accuracy: 0.9071


[0.2853456139564514, 0.9071000218391418]